In [ ]:
import altair as alt
import pandas as pd
from vega_datasets import data
alt.data_transformers.disable_max_rows()

In [ ]:
df = pd.read_csv("https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/ufo-scrubbed-geocoded-time-standardized-00.csv",
                 names=["datetime", "city", "state", "country", "shape","duration_seconds", "duration_hours_min", "comments","date_posted", "latitude", "longitude"], 
                 header=0)

df['year'] = pd.to_datetime(df['datetime'], errors='coerce').dt.year
df_filtered = df.dropna(subset=['year']).copy()
df_filtered['year'] = df_filtered['year'].astype(int)
yearly_counts = df_filtered.groupby('year').size().reset_index(name='count')
year_click = alt.selection_point(fields=['year'], name='Year')

In [ ]:
chart1 = alt.Chart(yearly_counts).mark_line(point=True).encode(
    x=alt.X('year:O', title='Year'),
    y=alt.Y('count:Q', title='Number of UFO Reports'),
    tooltip=['year', 'count']
).add_params(
    year_click
).properties(
    width=700,
    height=300,
    title='Number of UFO Reports per Year (click a point to see details)'
)

In [ ]:
columns_to_show = ['datetime', 'city', 'state', 'country', 'shape', 'duration_seconds', 'comments']


df_filtered['summary'] = (
    "[" + df_filtered['datetime'].astype(str) + "] "
    + df_filtered['shape'].astype(str) + " in " + df_filtered['city'].astype(str)
    + ", " + df_filtered['duration_seconds'].astype(str) + " sec"
)


# 选择需要的列（前 20 条）
table_clean = alt.Chart(df_filtered).mark_text(align='left', dx=3).encode(
    y=alt.Y('row_number:O', axis=None),
    text='summary:N',
    tooltip=['datetime', 'city', 'state', 'country', 'shape', 'duration_seconds', 'comments']
).transform_filter(
    year_click
).transform_window(
    row_number='row_number()'
).transform_filter(
    alt.datum.row_number < 20
).properties(
    width=700,
    height=400,
    title='Click a Year to See 20 Sample UFO Reports Below'
)



In [ ]:
final_chart = alt.vconcat(chart1, table_clean).configure_view(stroke=None)
final_chart.display()

**chart2**

In [ ]:
state_fips = {
    'al': '01', 'ak': '02', 'az': '04', 'ar': '05', 'ca': '06',
    'co': '08', 'ct': '09', 'de': '10', 'dc': '11', 'fl': '12',
    'ga': '13', 'hi': '15', 'id': '16', 'il': '17', 'in': '18',
    'ia': '19', 'ks': '20', 'ky': '21', 'la': '22', 'me': '23',
    'md': '24', 'ma': '25', 'mi': '26', 'mn': '27', 'ms': '28',
    'mo': '29', 'mt': '30', 'ne': '31', 'nv': '32', 'nh': '33',
    'nj': '34', 'nm': '35', 'ny': '36', 'nc': '37', 'nd': '38',
    'oh': '39', 'ok': '40', 'or': '41', 'pa': '42', 'ri': '44',
    'sc': '45', 'sd': '46', 'tn': '47', 'tx': '48', 'ut': '49',
    'vt': '50', 'va': '51', 'wa': '53', 'wv': '54', 'wi': '55',
    'wy': '56'
}

In [ ]:
df_geo = df.dropna(subset=['state']).copy()
state_counts = df_geo.groupby('state').size().reset_index(name='count')
state_counts['id'] = state_counts['state'].map(state_fips)
state_counts = state_counts.dropna(subset=['id']) 
state_counts['id'] = state_counts['id'].astype(str).str.zfill(2)

us_states = alt.topo_feature('https://cdn.jsdelivr.net/npm/us-atlas@3/states-10m.json', 'states')

In [ ]:
chart2 = alt.Chart(us_states).mark_geoshape(
    stroke='white'
).encode(
    color=alt.Color('count:Q', scale=alt.Scale(scheme='blues'), title='UFO Reports'),
    tooltip=[
        alt.Tooltip('id:N', title='State FIPS'),
        alt.Tooltip('count:Q', title='Report Count')
    ]
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(state_counts, 'id', ['count'])
).project(
    type='albersUsa'
).properties(
    title='UFO Sightings by State (Total Count)',
    width=700,
    height=450
)

chart2.display()